In [2]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint

# Import API key
from doe_api import doe_key

In [3]:
#demographics

url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json'
params1 = {
    'api_key': doe_key,
    'per_page': 250,
    'fields': "id,location.lon,location.lat,latest.school.name,"+
    
    "latest.root.location.lon,latest.root.location,latest.school.zip,latest.student.size,latest.root.id,latest.school.region_id,"+
    #"latest.aid.pell_grant_rate,latest.school.faculty_salary,latest.school.ft_faculty_rate,"+
    
    #admissions test requirements
    "latest.admissions.test_requirements,"+
    'latest.admissions.admission_rate.overall,'+
    
    #completion rate demographics
    'latest.completion.completion_rate_4yr_150nt,'+
    #"latest.completion.completion_rate_4yr_150_nhpi,"+ 
    #"latest.completion.completion_rate_4yr_150_aian,latest.completion.completion_rate_4yr_150_asian,"+
    #"latest.completion.completion_rate_4yr_150_hispanic,latest.completion.completion_rate_4yr_150_black,"+
    #"latest.completion.completion_rate_4yr_150_white,"+
    
    
    #student demographics
    "latest.student.demographics.race_ethnicity.white,latest.student.demographics.race_ethnicity.hispanic,"+
    "latest.student.demographics.race_ethnicity.black,latest.student.demographics.race_ethnicity.api,"+
    "latest.student.demographics.race_ethnicity.asian,"+
    "latest.student.demographics.race_ethnicity.aian,latest.student.demographics.women,latest.student.demographics.men,"+
    
    #median debt per demographic
    #"latest.aid.median_debt.income.30001_75000,latest.aid.median_debt.income.0_30000,latest.aid.median_debt.income.greater_than_75000,"+
    #"latest.aid.median_debt.noncompleters,latest.aid.median_debt.completers.overall,latest.aid.median_debt.male_students,"+
    #"latest.aid.median_debt.independent_students,latest.aid.median_debt.female_students,latest.aid.median_debt.dependent_students,"+
    
    #religious affiliation
    "latest.school.religious_affiliation,"+
    
    #undergrads age 25 and above
    "latest.student.share_25_older"+
    
    #page marker
    'page=0'
    
    
}



In [3]:
# Initialize an empty list to store the data
demographics_data = []
#starting page is 0
params1['page'] = 0

#loop that will loop through the page until we reach page 6
while params1['page'] < 7:
    #api call
    response = requests.get(url, params=params1).json()
    next_page_results = response.get("results", [])
    demographics_data.extend(next_page_results)
    #to go to the next page of results
    params1['page'] += 1

In [4]:
#create dataframe
demographics_df = pd.DataFrame(demographics_data)
#fill na columns to 0
demographics_df = demographics_df.fillna(0)
#drop unwanted columns
demographics_df = demographics_df.drop(columns = ['latest.student.demographics.race_ethnicity.api_2000',
                                                 'latest.school.region_id',
                                                  'latest.school.religious_affiliation',
                                                 'latest.school.zip'])

In [5]:
#rename columns
demographics_clean = demographics_df.rename(columns={
    'id':'School_Id',
    'location.lat':'School Latitude',
    'location.lon':'School Longitude',
    'latest.school.name':'School Name',
    'latest.student.size':'Student Count',
    'latest.admissions.test_requirements':'Standardized Test required? (1:Required, 2:Recommended, 3:Neither required nor recommended , 4:Do not know, 5: Considered but not required)',
    'latest.admissions.admission_rate.overall': 'Admission Rate',
    'latest.completion.completion_rate_4yr_150nt':'4Yr Uni Completion Rates',
    'latest.student.demographics.race_ethnicity.white': '% White Students',
    'latest.student.demographics.race_ethnicity.hispanic':'% Hispanic Students',
    'latest.student.demographics.race_ethnicity.black': '% Black Students',
    'latest.student.demographics.race_ethnicity.asian': '% Asian Students',
    'latest.student.demographics.race_ethnicity.aian': '% AIAN Students',
    'latest.student.demographics.women':'% Female Students',
    'latest.student.demographics.men':'% Male Students'})

In [6]:
#for loop to multiply the list of columns by 100 for the percent
columns_percent = ['Admission Rate','4Yr Uni Completion Rates','% White Students',
                  '% Hispanic Students','% Black Students','% Asian Students','% AIAN Students',
                  '% Female Students', '% Male Students']
for columns in columns_percent:
    demographics_clean[columns] = demographics_clean[columns]*100

In [7]:
#reorder columns
demographics_clean = demographics_clean[['School_Id','School Name','Student Count',
'Standardized Test required? (1:Required, 2:Recommended, 3:Neither required nor recommended , 4:Do not know, 5: Considered but not required)',
'Admission Rate', '4Yr Uni Completion Rates', '% White Students',
'% Hispanic Students', '% Black Students','% Asian Students', '% AIAN Students','% Female Students', '% Male Students',
'School Latitude','School Longitude']]

In [8]:
demographics_clean = demographics_clean.drop_duplicates(subset='School Name')

In [9]:
demographics_clean = demographics_clean.set_index('School Name')

In [10]:
#demographics_clean.to_csv('data/demographics.csv', index=False, header=True)
#demographics_clean.to_json('data/demographics_clean.json', orient = 'index', compression = 'infer', index = 'true')

In [9]:
#male/female
url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json'
params3 = {
    'api_key': doe_key,
    'per_page': 250,
    'fields': "id,location.lon,location.lat,latest.school.name,"+
    
    #male 
    #general completion
    "latest.completion.completion_rate_4yr_150nt_pooled,"+
    
    "latest.completion.title_iv.male.completed_by.2yrs,latest.completion.title_iv.male.completed_by.3yrs,"+
    "latest.completion.title_iv.male.completed_by.4yrs,latest.completion.title_iv.male.completed_by.6yrs,"+
    "latest.completion.title_iv.male.completed_by.8yrs,"+
    #general withdrew
    "latest.completion.title_iv.male.withdrawn_by.2yrs,latest.completion.title_iv.male.withdrawn_by.3yrs,"+
    "latest.completion.title_iv.male.withdrawn_by.4yrs,latest.completion.title_iv.male.withdrawn_by.6yrs,"+
    "latest.completion.title_iv.male.withdrawn_by.8yrs,"+
    
    
    #female 
    #general completion
    "latest.completion.title_iv.female.completed_by.2yrs,latest.completion.title_iv.female.completed_by.3yrs,"+
    "latest.completion.title_iv.female.completed_by.4yrs,latest.completion.title_iv.female.completed_by.6yrs,"+
    "latest.completion.title_iv.female.completed_by.8yrs,"+
    #general withdrew
    "latest.completion.title_iv.female.withdrawn_by.2yrs,latest.completion.title_iv.female.withdrawn_by.3yrs,"+
    "latest.completion.title_iv.female.withdrawn_by.4yrs,latest.completion.title_iv.female.withdrawn_by.6yrs,"+
    "latest.completion.title_iv.female.withdrawn_by.8yrs,"+
    
    #page marker
    'page=0'
    
}

In [10]:
# Initialize an empty list to store the data
gender_completion = []
#starting page is 0
params3['page'] = 0

#loop that will loop through the page until we reach page 6
while params3['page'] < 7:
    #api call
    response = requests.get(url, params=params3).json()
    next_page_results = response.get("results", [])
    gender_completion.extend(next_page_results)
    #to go to the next page of results
    params3['page'] += 1

In [11]:
#create dataframe
gender_completion_df = pd.DataFrame(gender_completion)
#fill nan values to 0
gender_completion_df = gender_completion_df.fillna(0)
#rename columns
gender_completion_clean = gender_completion_df.rename(columns = {
        'id':'School_Id',
        'location.lat':'School Latitude',
        'location.lon':'School Longitude',
        'latest.school.name': 'School Name',
       'latest.completion.title_iv.male.completed_by.2yrs':'% Male Students Completed within 2yrs',
       'latest.completion.title_iv.male.completed_by.3yrs':'% Male Students Completed within 3yrs',
       'latest.completion.title_iv.male.completed_by.4yrs':'% Male Students Completed within 4yrs',
       'latest.completion.title_iv.male.completed_by.6yrs':'% Male Students Completed within 6yrs',
       'latest.completion.title_iv.male.completed_by.8yrs':'% Male Students Completed within 8yrs',
       'latest.completion.title_iv.male.withdrawn_by.2yrs':'% Male Students Withdrawn by 2yrs',
       'latest.completion.title_iv.male.withdrawn_by.3yrs':'% Male Students Withdrawn by 3yrs',
       'latest.completion.title_iv.male.withdrawn_by.4yrs':'% Male Students Withdrawn by 4yrs',
       'latest.completion.title_iv.male.withdrawn_by.6yrs':'% Male Students Withdrawn by 6yrs',
       'latest.completion.title_iv.male.withdrawn_by.8yrs':'% Male Students Withdrawn by 8yrs',
       'latest.completion.title_iv.female.completed_by.2yrs':'% Female Students Completed within 2yrs',
       'latest.completion.title_iv.female.completed_by.3yrs':'% Female Students Completed within 3yrs',
       'latest.completion.title_iv.female.completed_by.4yrs':'% Female Students Completed within 4yrs',
       'latest.completion.title_iv.female.completed_by.6yrs':'% Female Students Completed within 6yrs',
       'latest.completion.title_iv.female.completed_by.8yrs':'% Female Students Completed within 8yrs',
       'latest.completion.title_iv.female.withdrawn_by.2yrs':'% Female Students Withdrawn by 2yrs',
       'latest.completion.title_iv.female.withdrawn_by.3yrs':'% Female Students Withdrawn by 3yrs',
       'latest.completion.title_iv.female.withdrawn_by.4yrs':'% Female Students Withdrawn by 4yrs',
       'latest.completion.title_iv.female.withdrawn_by.6yrs':'% Female Students Withdrawn by 6yrs',
       'latest.completion.title_iv.female.withdrawn_by.8yrs':'% Female Students Withdrawn by 8yrs'
    
})

In [13]:
gender_completion_clean

,School Name,latest.completion.completion_rate_4yr_150nt_pooled,% Male Students Completed within 2yrs,% Male Students Completed within 3yrs,% Male Students Completed within 4yrs,% Male Students Completed within 6yrs,% Male Students Completed within 8yrs,% Male Students Withdrawn by 2yrs,% Male Students Withdrawn by 3yrs,% Male Students Withdrawn by 4yrs,...,% Female Students Completed within 6yrs,% Female Students Completed within 8yrs,% Female Students Withdrawn by 2yrs,% Female Students Withdrawn by 3yrs,% Female Students Withdrawn by 4yrs,% Female Students Withdrawn by 6yrs,% Female Students Withdrawn by 8yrs,School_Id,School Latitude,School Longitude
0,Alabama A & M University,0.2833,0.040663,0.081267,0.176385,0.244556,0.255682,0.373494,0.353994,0.424198,...,0.357480,0.318015,0.297149,0.251813,0.287559,0.233071,0.196691,100654,34.783368,-86.568502
1,University of Alabama at Birmingham,0.6181,0.136760,0.288678,0.425299,0.509948,0.503590,0.223868,0.263613,0.248008,...,0.558140,0.569100,0.189072,0.192921,0.186350,0.150849,0.123207,100663,33.505697,-86.799345
2,Amridge University,0.3846,0.000000,0.000000,0.000000,0.000000,0.272727,0.600000,0.500000,0.489796,...,0.000000,0.123894,0.508065,0.525253,0.518987,0.495413,0.283186,100690,32.362609,-86.174010
3,University of Alabama in Huntsville,0.5925,0.062350,0.235294,0.346726,0.481054,0.476274,0.219424,0.263427,0.313988,...,0.480080,0.510169,0.198020,0.178964,0.231511,0.145418,0.101695,100706,34.724557,-86.640449
4,Alabama State University,0.3015,0.054502,0.087866,0.194499,0.250000,0.223011,0.398104,0.464435,0.388998,...,0.368715,0.322953,0.284286,0.316770,0.274933,0.216946,0.198385,100724,32.364317,-86.295677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Robert Morgan Educational Center and Technical...,0.0000,0.368421,0.000000,0.000000,0.182482,0.267327,0.484211,0.632075,0.743119,...,0.418033,0.333333,0.423529,0.500000,0.355556,0.262295,0.000000,136826,25.599247,-80.390054
696,Rollins College,0.7590,0.207792,0.588652,0.654762,0.642105,0.666667,0.097403,0.070922,0.119048,...,0.741445,0.708709,0.077441,0.072131,0.080769,0.098859,0.099099,136950,28.592787,-81.349239
697,First Coast Technical College,0.0000,0.736842,0.590909,0.000000,0.541353,0.402062,0.000000,0.295455,0.500000,...,0.588235,0.497382,0.000000,0.287770,0.258333,0.217647,0.094241,137023,29.924377,-81.345829
698,Saint Leo University,0.4830,0.172117,0.268953,0.300317,0.397849,0.445068,0.387263,0.370939,0.353016,...,0.362931,0.402647,0.335386,0.286657,0.310016,0.239988,0.197267,137032,28.336776,-82.257801


In [14]:
gender_completion_clean = gender_completion_clean[['School_Id','School Name', 
        '% Male Students Completed within 2yrs',
       '% Male Students Completed within 3yrs',
       '% Male Students Completed within 4yrs',
       '% Male Students Completed within 6yrs',
       '% Male Students Completed within 8yrs',
       '% Male Students Withdrawn by 2yrs',
       '% Male Students Withdrawn by 3yrs',
       '% Male Students Withdrawn by 4yrs',
       '% Male Students Withdrawn by 6yrs',
       '% Male Students Withdrawn by 8yrs',
       '% Female Students Completed within 2yrs',
       '% Female Students Completed within 3yrs',
       '% Female Students Completed within 4yrs',
       '% Female Students Completed within 6yrs',
       '% Female Students Completed within 8yrs',
       '% Female Students Withdrawn by 2yrs',
       '% Female Students Withdrawn by 3yrs',
       '% Female Students Withdrawn by 4yrs',
       '% Female Students Withdrawn by 6yrs',
       '% Female Students Withdrawn by 8yrs']]

In [15]:
#for loop to convert floats to percentage
for i in gender_completion_clean.columns[2:]:
    gender_completion_clean[i] = gender_completion_clean[i] * 100

In [16]:
gender_completion_clean = gender_completion_clean.drop_duplicates(subset='School Name')

In [17]:
gender_completion_clean = gender_completion_clean.set_index('School Name')

In [18]:
#gender_completion_clean.to_csv('data/gender_completion.csv', index=False, header=True)

In [19]:
#gender_completion_clean.to_json('data/gender_completion_clean.json', orient = 'index', compression = 'infer', index = 'true')

In [20]:
#gender_completion_clean['School Name'].astype(str).to_json('data/schools.json', orient = 'values', compression = 'infer', index = 'true')